In [1]:
# pip install pandas

In [2]:
# pip install gensim

In [3]:
# pip install scipy

In [4]:
# pip install pyLDAvis

In [1]:
import pandas as pd
import numpy as np
import glob
from nltk.corpus import stopwords


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
from nltk.corpus import stopwords

# vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
df = pd.read_parquet('herb.parquet')
df

,id,title,abstract,pub_year,authors_list,relative_date,keyword,link
0,Curcuma_longa_1,"Curcuma longa (Turmeric): Ethnomedicinal Uses,...","Introduction: Curcuma longa, popularly known a...",2023,"['Emeka J. Iweala', 'Emeka J. Iweala', 'Miracl...",6/5/2024 15:59,Curcuma_longa,https://www.researchgate.net/publication/36733...
1,Curcuma_longa_2,Biological activities of Curcuma longa L,There are several data in the literature indic...,2001,"['C.A.C. Araújo', 'C.A.C. Araújo', 'L L Leon',...",6/5/2024 15:59,Curcuma_longa,https://www.researchgate.net/publication/11843...
2,Curcuma_longa_3,Chemical constituents and biological activitie...,"Turmeric (Curcuma longa) is used as spice, pre...",2008,"['Abhishek Niranjan', 'National Botanical Rese...",6/5/2024 15:59,Curcuma_longa,https://www.researchgate.net/publication/28386...
3,Curcuma_longa_4,Anti-cancer activity of Curcuma longa linn.(Tu...,Curcuma longa Linn. (Turmeric) is used extensi...,2011,"['Annapurna A', 'Annapurna A', 'Suhasin Ganta'...",6/6/2024 4:13,Curcuma_longa,https://www.researchgate.net/publication/23080...
4,Curcuma_longa_5,Chemical Composition and Product Quality Contr...,Chemical constituents of various tissues of tu...,2011,"['Shiyou Li', 'Stephen F. Austin State Univers...",6/6/2024 1:16,Curcuma_longa,https://www.researchgate.net/publication/27009...
...,...,...,...,...,...,...,...,...
1704,Lysiphyllum_strychnifolium_49,Challenges in Optimizing Nanoplatforms Used fo...,"In this study, we focused on innovative approa...",2024,"['Dorin Ioan Cocoș', 'Dorin Ioan Cocoș', 'O.D....",6/5/2024 16:18,Lysiphyllum_strychnifolium,https://www.researchgate.net/publication/38040...
1705,Lysiphyllum_strychnifolium_50,Effect of lowering postprandial hyperglycemia ...,"Glucose tolerance declines with age, resulting...",2004,"['Annette M Chang', 'Annette M Chang', 'Marla ...",6/5/2024 16:18,Lysiphyllum_strychnifolium,https://www.researchgate.net/publication/84953...
1706,Lysiphyllum_strychnifolium_51,"TM4: A Free, Open-Source System for Microarray...",Microarrays have emerged as the\r\npremier too...,2003,"['A.I. Saeed', 'A.I. Saeed', 'Vasily Sharov', ...",6/5/2024 16:18,Lysiphyllum_strychnifolium,https://www.researchgate.net/publication/10877...
1707,Lysiphyllum_strychnifolium_52,Automedicación y uso de plantas medicinales en...,Introducción:\r\nLa expansión acelerada del SA...,2023,"['Piero Pasache Pinedo', 'Piero Pasache Pinedo...",6/5/2024 16:18,Lysiphyllum_strychnifolium,https://www.researchgate.net/publication/37113...


In [5]:
stopwords = stopwords.words("english")

In [7]:
# print (stopwords)

In [11]:
data = df['abstract']
data[0][0:90]

'Introduction: Curcuma longa, popularly known as Turmeric, is a rhizomatous herbaceous pere'

In [13]:
# Filter for rows with non-null values in both title and abstract
df_filtered = df[~df['abstract'].isnull()]
data = df_filtered['abstract']

In [15]:
# Load spaCy model (disable unnecessary components for efficiency)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out

# Lemmatize the combined titles and abstracts
lemmatized_texts = lemmatization(data.tolist())

# Print the first 90 characters of the first lemmatized text
print(lemmatized_texts[0][0:90])

introduction longa popularly know rhizomatous herbaceous perennial plant use folk medicine


In [16]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['introduction', 'longa', 'popularly', 'know', 'rhizomatous', 'herbaceous', 'perennial', 'plant', 'use', 'folk', 'medicine', 'treatment', 'prevention', 'management', 'various', 'illness', 'such', 'cancer', 'diabete', 'diarrhoea']


In [17]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 3), (2, 1), (3, 1), (4, 4), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 4), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 2), (19, 1)]
action


In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=38,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.354796 -0.359812       1        1  20.949242
10    -0.229488 -0.428363       2        1   9.933320
3     -0.106283 -0.457121       3        1   7.750813
7     -0.389311 -0.269701       4        1   6.760784
26    -0.292779 -0.368624       5        1   6.653781
16    -0.494243 -0.040538       6        1   5.991920
31     0.252645 -0.421325       7        1   5.088376
1      0.014230 -0.475707       8        1   4.586289
21     0.385397 -0.180391       9        1   2.769458
20     0.126185 -0.400210      10        1   2.686093
19    -0.428460 -0.083660      11        1   2.418173
17    -0.034003  0.461348      12        1   2.330598
0      0.332052  0.346184      13        1   2.112827
15     0.204256 -0.249101      14        1   2.078375
11     0.446671  0.074234      15        1   1.755195
4      0.437214 -0.060610      16        1   1.568551
33    -0.097055 -0.246298      17        1   1.534372
14    -0.327047  0.306885      18        1   1.399109
25     0.138510  0.446458      19        1   1.318179
8     -0.408328  0.158883      20        1   1.148232
23    -0.301509 -0.098179      21        1   1.136937
32     0.302417  0.241426      22        1   0.985222
18    -0.136342  0.380160      23        1   0.895413
30    -0.199115  0.293762      24        1   0.868157
12     0.325914  0.129905      25        1   0.858688
29     0.124407  0.326585      26        1   0.839272
9     -0.272966  0.162990      27        1   0.717086
24     0.024297 -0.097035      28        1   0.552956
2     -0.159559  0.024765      29        1   0.487249
28     0.245096 -0.003072      30        1   0.409726
35    -0.030907  0.244518      31        1   0.382081
22    -0.055492  0.080779      32        1   0.347219
36     0.114271  0.150837      33        1   0.266602
6      0.096965 -0.001367      34        1   0.241631
5     -0.000448  0.129282      35        1   0.123642
34     0.019986  0.095298      36        1   0.039404
27     0.009015  0.093408      37        1   0.007534
37     0.009015  0.093408      38        1   0.007493, topic_info=              Term         Freq        Total Category  logprob  loglift
105        extract  2840.000000  2840.000000  Default    10.00  10.0000
64           plant  1861.000000  1861.000000  Default     9.00   9.0000
136           cell  1344.000000  1344.000000  Default     8.00   8.0000
1         activity  2443.000000  2443.000000  Default     7.00   7.0000
280            oil  1013.000000  1013.000000  Default     6.00   6.0000
..             ...          ...          ...      ...      ...      ...
4             anti     0.001153   860.787209  Topic38    -9.34  -4.0244
5       anticancer     0.001153   113.684487  Topic38    -9.34  -2.0000
6     antidiabetic     0.001153    23.466128  Topic38    -9.34  -0.4221
8    applicability     0.001153     7.120859  Topic38    -9.34   0.7704
9      application     0.001153   311.879351  Topic38    -9.34  -3.0091

[571 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
1729      5  0.996175       acarbose
2022      2  0.987182         accord
237      25  0.981745     accumulate
2023     33  0.989675   accumulation
1165     27  0.958028         acetic
...     ...       ...            ...
345       9  0.906496          yield
345      15  0.088583          yield
798      23  0.966146  zingiberaceae
1324     24  0.976866           zone
1184      2  0.992818             μg

[775 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 11, 4, 8, 27, 17, 32, 2, 22, 21, 20, 18, 1, 16, 12, 5, 34, 15, 26, 9, 24, 33, 19, 31, 13, 30, 10, 25, 3, 29, 36, 23, 37, 7, 6, 35, 28, 38])

In [27]:
num_topics = lda_model.num_topics
words_list = []

# Extract words for each topic
for i in range(num_topics):
    topic_words = lda_model.show_topic(i, topn=10)  # Adjust topn as needed
    for word, prob in topic_words:
        words_list.append({'Topic': f'Topic_{i+1}', 'Word': word})

# Create a DataFrame
df = pd.DataFrame(words_list)

# Save to CSV
df.to_csv('topics_words_herbs.csv', index=False)

df

,Topic,Word
0,Topic_1,cell
1,Topic_1,cancer
2,Topic_1,apoptosis
3,Topic_1,line
4,Topic_1,induce
...,...,...
375,Topic_38,grandi
376,Topic_38,litsericinone
377,Topic_38,valerianoide
378,Topic_38,fissitungfine
